[View in Colaboratory](https://colab.research.google.com/github/francoisjaulin/Labelling_unlabelled/blob/master/LONDON_ICU_DATATHON.ipynb)

In [6]:
# Setup
%set_env GOOGLE_CLOUD_PROJECT physionet-data

from google.colab import auth
auth.authenticate_user()
!touch ~/.bigqueryrc

# Silence some log spam from this library.`
import logging
logging.getLogger('googleapiclient').setLevel(logging.CRITICAL)
    


env: GOOGLE_CLOUD_PROJECT=physionet-data


In [7]:
!pip install sklearn-pandas

from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [22]:
!pip install dialogflow
#import dialogflow

    100% |████████████████████████████████| 204kB 4.7MB/s 
  Running setup.py bdist_wheel for dialogflow ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/65/36/05/cf539b1a37c148446f433be31cb625b6f65c193c847a67a5ca
Successfully built dialogflow


In [0]:
#import dialogflow
def detect_intent_texts(london_datathon, session_id, texts, language_code):
    """Returns the result of detect intent with texts as inputs.

    Using the same `session_id` between requests allows continuation
    of the conversaion."""
    import dialogflow_v2 as dialogflow
    session_client = dialogflow.SessionsClient()

    session = session_client.session_path(project_id, session_id)
    print('Session path: {}\n'.format(session))

    for text in texts:
        text_input = dialogflow.types.TextInput(
            text=text, language_code=language_code)

        query_input = dialogflow.types.QueryInput(text=text_input)

        response = session_client.detect_intent(
            session=session, query_input=query_input)

        print('=' * 20)
        print('Query text: {}'.format(response.query_result.query_text))
        print('Detected intent: {} (confidence: {})\n'.format(
            response.query_result.intent.display_name,
            response.query_result.intent_detection_confidence))
        print('Fulfillment text: {}\n'.format(
            response.query_result.fulfillment_text))


In [0]:
detect_intent_text()

In [16]:
# Query to obtain the labels
query = '''
SELECT
  ARRAY_AGG(value_as_string) as values,
  person_id,
  visit_occurrence_id
FROM
  `chc-mimic-data.mimic3_aphp.observation`  AS t
WHERE
  t.observation_source_concept_id='2001028679'
  AND person_id < %d
GROUP BY
  person_id,
  visit_occurrence_id
'''

# query = 'SELECT * FROM [physionet-data:mimiciii_notes.noteevents] LIMIT 1000'


query = (query) % max_id

df = pd.read_gbq(query, project_id='london-datathon', verbose=False, dialect='legacy')
df.head(20)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,653935,93788,149363,2189-02-06,2189-02-06 16:35:00,2189-02-06 16:36:01,Physician,CVI,15626,None,TITLE:\n CVICU\n HPI:\n HD3\n [**2-6**...
1,440464,84747,163652,2121-03-05,2121-03-05 20:09:00,2121-03-05 20:09:22,Physician,CVI,15626,None,TITLE:\n SICU\n HPI:\n 54yoW s/p CABGx3(...
2,649960,86918,135138,2138-12-11,2138-12-11 17:37:00,2138-12-11 17:43:47,Physician,CVI,15626,None,TITLE:\n CVICU\n HPI:\n 66F s/p MVR(#31 ...
3,1040017,26533,None,2153-09-10,2153-09-10 00:00:00,NaT,Radiology,RVG,None,None,RVG ...
4,824711,13725,None,2172-06-16,2172-06-16 00:00:00,NaT,Radiology,RVG,None,None,RVG ...
5,849283,4058,None,2140-12-28,2140-12-28 00:00:00,NaT,Radiology,RVG,None,None,RVG ...
6,801633,26688,132950,2173-08-20,2173-08-20 00:00:00,NaT,Radiology,RVG,None,None,RVG ...
7,873692,18148,None,2149-06-30,2149-06-30 00:00:00,NaT,Radiology,RVG,None,None,RVG ...
8,613274,79836,102516,2129-01-25,2129-01-25 11:26:00,2129-01-25 11:26:35,Physician,CVICU,15626,None,CVICU\n HPI:\n HD2\n Pre-Op Weight:0 lbs...
9,630353,73150,130027,2101-05-13,2101-05-13 11:48:00,2101-05-13 11:48:40,Physician,CVICU,15626,None,TITLE:\n CVICU\n HPI:\n HD4 POD 3-redo...


In [17]:
label_name = 'tobacco'

labels = {}

positives = 0
total = 0
for index, row in df.iterrows():
  key = str(row["person_id"]) + ":" + str(row["visit_occurrence_id"])
  label_values = row["values"]
  conditions = []
  for v in label_values:
    conditions.append(v['v'])
  label_value = (label_name in conditions)
  if label_value:
    positives = positives + 1
  total = total + 1
  labels[key] = label_value

print ('positives %d' % positives)
print ('total %d' % total)
print (labels)

KeyError: ignored

In [0]:
# Select the discharge summary medical history
query = 'SELECT nlp.note_id,nlp.lexical_variant,note.person_id,note.visit_occurrence_id FROM `chc-mimic-data.mimic3_aphp.note_nlp` as nlp inner join `chc-mimic-data.mimic3_aphp.note` as note ON nlp.note_id = note.note_id WHERE nlp.lexical_variant LIKE "HIST%" AND note.person_id < {maxid}'.format(maxid=max_id)
df = pd.read_gbq(query, project_id='chc-mimic-analysis', verbose=False, dialect='standard')



In [0]:
# Dump some to see what it looks like
print (len(df['note_id']))
df['key'] = df['person_id'].astype(str) + ':' + df['visit_occurrence_id'].astype(str)
df.head(3)

NameError: ignored

In [0]:

coarse_label = labels

NameError: ignored

In [0]:
# Print the top tokens having the most influence
# when the model considers a particular item
# ie, the largest (by absolute value) products of scalars
# that compose the dot product in the linear model
def print_top_tokens_decision(model, item_weights, N=10):
  import operator
  
  model_coefs = model.coef_[0]
  
  coefs = [ abs(model_coefs*item_weights) for model_coefs, item_weights in zip(model_coefs, item_weights) ]
  
  index_coef = sorted( enumerate(coefs), key=operator.itemgetter(1))
  model_size = len(index_coef)
  print (index_coef[-1])
  index_token = { index:token for token, index in vectorizer.vocabulary_.items() }
  largest_index_list, largest_coef_list = zip( *index_coef )
  largest_index_list = largest_index_list[-N:]
  # print (largest_index_list)
  # print( largest_coef_list[-N:])
  print ('Top {N} tokens (for specific item):'.format(N=N))
  print ([index_token[index] for index in largest_index_list])
  print ([coefs[index] for index in largest_index_list])
  

In [0]:
from sklearn_pandas import DataFrameMapper
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import GaussianRandomProjection
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Create the feacture vector for the notes.
NGRAM_MAX = 3
vectorizer = CountVectorizer(min_df=10, stop_words='english', ngram_range=(1,NGRAM_MAX))

tfidf_mapper = DataFrameMapper([
   ('lexical_variant', [vectorizer,TfidfTransformer()] )
   ])
tfidf = tfidf_mapper.fit_transform(df)


# Make the labels.
label_manual = {
    "62064720:None" : 0,
    "62064225:None" : 0,
    "62065246:1331" : 1,
    "62064225:1792" : 1,
    "62063486:1218" : 1,
    
               }
label = dict( list(coarse_label.items()) + list(label_manual.items()) )

count = len(df.note_id)

labels = np.zeros(count)

# Make the labels from the positives found.
for i in range(0, count):
 nid = df.key[i]
 if nid in label:
   labels[i] = label[nid]
  
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Train a model for the active learning.
#model = SGDClassifier(loss='hinge', penalty='none',
#                     alpha=0.00, random_state=42,
#                     learning_rate='constant',
#                     eta0=1.0, fit_intercept=False, class_weight='balanced')
model = LogisticRegression(max_iter=1, fit_intercept=False)
#model = RandomForestClassifier(max_depth=8, min_samples_leaf=2)
model.fit(tfidf, labels)
predictions = model.predict_proba(tfidf)

def plot_histogram(values, title):
  
  import matplotlib.mlab as mlab
  import matplotlib.pyplot as plt

  # the histogram of the data
  n, bins, patches = plt.hist(values, 50, normed=1, facecolor='green', alpha=0.75)

  # plt.xlabel('Smarts')
  # plt.ylabel('Probability')
  plt.title( title )
  # plt.axis([40, 160, 0, 0.03])
  plt.grid(True)

  plt.show()

plot_histogram( predictions, 'predictions' )
  
# Print the top tokens
def print_top_tokens_model(model, N=10):
  import operator
  coefs = model.coef_[0]
  index_coef = sorted( enumerate(coefs), key=operator.itemgetter(1))
  model_size = len(index_coef)
  print (index_coef[-1])
  index_token = { index:token for token, index in vectorizer.vocabulary_.items() }
  largest_index_list, largest_coef_list = zip( *index_coef )
  largest_index_list = largest_index_list[-N:]
  # print (largest_index_list)
  # print( largest_coef_list[-N:])
  print ('Top {N} tokens (overall):'.format(N=N))
  print ([index_token[index] for index in largest_index_list])
  
print_top_tokens_model(model, N=10) 

  
# Find the next sample for active learning

policy = "uncertain"
#policy = "largest"

if policy == "uncertain":
  candidate = -1
  candidate_score = 100
  for i in range(0, count):
    if df.key[i] not in label:
       score = abs(predictions[i][0] - 0.5)
       if score < candidate_score:
         candidate_score = score
         candidate = i
        
if policy == "largest":
  candidate = -1
  candidate_score = 0
  for i in range(0, count):
    if df.key[i] not in label:
       score = predictions[i][0]
       if score > candidate_score:
         candidate_score = score
         candidate = i
      
#print (labels[:10])
#print (predictions[:10])
print ('next candidate is key %s note_id %d with score %s' % (df.key[candidate],
                                                              df.note_id[candidate],
                                                              candidate_score))

print_top_tokens_decision(model, tfidf[candidate], N=10)
print (df.lexical_variant[candidate])



ModuleNotFoundError: ignored

In [0]:
x = tfidf[candidate]
print(x)





NameError: ignored